<a href="https://colab.research.google.com/github/BogdanT54/Cicluri-Economice-si-Randamentul-Pietelor/blob/main/Analiza_Active_Licenta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install yfinance statsmodels arch seaborn

In [ ]:
# Importuri
from datetime import timedelta
import yfinance as yf
import pandas as pd
from scipy.stats import skew, kurtosis
import matplotlib.pyplot as plt
import seaborn as sns
from arch import arch_model
import statsmodels.tsa.stattools as ts
from google.colab import files
from arch.unitroot import ADF, PhillipsPerron, KPSS
from statsmodels.stats.diagnostic import het_arch
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import matplotlib.dates as mdates
import warnings
from statsmodels.tsa.arima.model import ARIMA
from scipy.stats import kurtosis, skew
import numpy as np
import pandas as pd
import warnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
warnings.filterwarnings('ignore')

# **Citire, prelucrare si curatare date**

In [ ]:
#Fac un dataframe cu fisierul dat
sp500 = yf.download('^GSPC', start='2005-01-01', end='2025-12-30', interval='1d')
gold = yf.download('GC=F', start='2005-01-01', end='2025-12-30', interval='1d')
oil = yf.download('CL=F', start='2005-01-01', end='2025-12-30', interval='1d')

bitcoin = pd.read_csv ('Bitcoin.csv', index_col=0, parse_dates=True)


#Citire, curatare si prelucrare date ROTX
rotx = pd.read_csv('ROTX.csv', sep=';', index_col=0, parse_dates=True)
rotx = rotx.loc[:, ~rotx.columns.str.contains('^Unnamed')]
rotx.drop(columns=['Chg.%'], inplace=True)
rotx.rename(columns={'Last Close': 'Close'}, inplace=True)

#Verific Df-ul
bitcoin.head(-1)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,volume,marketCap
Period,,,,,,
2010-07-14,0.058157,0.061588,0.048647,0.056402,2.615400e+02,1.902596e+05
2010-07-15,0.056403,0.067954,0.053969,0.057568,4.458000e+02,1.959821e+05
2010-07-16,0.058001,0.072220,0.057484,0.066492,4.972500e+02,2.280474e+05
2010-07-17,0.066500,0.077735,0.057418,0.065993,1.999000e+01,2.269048e+05
2010-07-18,0.066088,0.080858,0.064221,0.078814,7.513000e+01,2.716692e+05
...,...,...,...,...,...,...
2025-05-20,105434.368300,105997.692300,104232.699900,104731.983300,4.454486e+10,2.081860e+12
2025-05-21,104750.780100,105936.690200,100436.885900,101575.952300,5.747930e+10,2.018780e+12
2025-05-22,101574.367100,105376.773400,101169.572400,104390.346900,4.885665e+10,2.074800e+12


In [ ]:
# Extragere coloana 'Close' și redenumire
sp500_close = sp500['Close']
gold_close = gold['Close']
oil_close = oil['Close']
bitcoin_close = bitcoin['Close']
rotx_close = rotx['Close']

# Concatenare
real_prices_df = pd.concat([sp500_close, gold_close, oil_close, bitcoin_close, rotx_close], axis=1)
real_prices_df.columns = ['SP500', 'Gold', 'Oil', 'Bitcoin', 'ROTX']

# Vizualizare
real_prices_df.head()

In [ ]:
# Prelucrare randamente + logaritmare cu backfill
sp500_log_returns = np.log(sp500 / sp500.shift(1)).replace([np.inf, -np.inf, 0], np.nan).dropna().bfill()
gold_log_returns = np.log(gold / gold.shift(1)).replace([np.inf, -np.inf, 0], np.nan).dropna().bfill()
oil_log_returns = np.log(oil / oil.shift(1)).replace([np.inf, -np.inf, 0], np.nan).dropna().bfill()
bitcoin_log_returns = np.log(bitcoin / bitcoin.shift(1)).bfill()
rotx_log_returns = np.log(rotx / rotx.shift(1)).replace([np.inf, -np.inf, 0], np.nan).dropna().bfill()


sp500_log_returns_close = sp500_log_returns['Close']
gold_log_returns_close = gold_log_returns['Close']
oil_log_returns_close = oil_log_returns['Close']
bitcoin_log_returns_close = bitcoin_log_returns['Close']
rotx_log_returns_close = rotx_log_returns['Close']

log_real_prices_df = pd.concat([sp500_log_returns_close, gold_log_returns_close, oil_log_returns_close, bitcoin_log_returns_close, rotx_log_returns_close], axis=1)
log_real_prices_df.columns = ['SP500', 'Gold', 'Oil', 'Bitcoin', 'ROTX']

log_real_prices_df.head(-1)

# **Analiza Descriptiva**

# **Multiple**

In [ ]:
import matplotlib.pyplot as plt

# Creez figura cu 3 rânduri și 2 coloane (ultimul subplot va rămâne gol)
fig, axs = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Evoluția Prețurilor - Active Majore (2005–2025)', fontsize=16)

# Așezăm activele în ordinea dorită: SP500, Gold, Bitcoin, ROTX, Oil
assets = ['SP500', 'ROTX', 'Bitcoin', 'Gold',]

# Flatten pentru accesare simplificată
axs = axs.flatten()

# Plot individual
for i, asset in enumerate(assets):
    axs[i].plot(real_prices_df.index, real_prices_df[asset])
    axs[i].set_title(asset)
    axs[i].tick_params(axis='x', rotation=45)
    axs[i].grid(True)

plt.tight_layout()
plt.subplots_adjust(top=0.93)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Creez figura cu 2 rânduri și 2 coloane
fig, axs = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Randamente Logaritmice - Active Majore (2005–2025)', fontsize=16)

# Aleg activele (fără Oil)
assets = ['SP500', 'ROTX', 'Bitcoin', 'Gold']

# Flatten pentru acces simplu
axs = axs.flatten()

# Plot individual
for i, asset in enumerate(assets):
    axs[i].plot(log_real_prices_df.index, log_real_prices_df[asset])
    axs[i].set_title(f"{asset} Log Returns")
    axs[i].tick_params(axis='x', rotation=45)
    axs[i].grid(True)

plt.tight_layout()
plt.subplots_adjust(top=0.93)
plt.show()


# **Individuale**

In [ ]:
#Bitcoin pret inchidere plot
fig, ax = plt.subplots(figsize=(20, 8))
bitcoin['Close'].plot(ax=ax, title='Bitcoin - Preț de închidere (2010–2025)', legend=False)
ax.set_ylabel("Preț (USD)")
ax.grid(True)
ax.tick_params(axis='x', rotation=45)
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
fig.tight_layout()
plt.show()


#Bitcoin log randamente
fig, ax = plt.subplots(figsize=(20, 8))
bitcoin_log_returns['Close'].plot(ax=ax, title='Bitcoin - Randamente zilnice log (2010–2025)', legend=False)
ax.set_ylabel("Randament")
ax.grid(True)
ax.tick_params(axis='x', rotation=45)
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
fig.tight_layout()
plt.show()

In [ ]:
#Gold plot
fig, ax = plt.subplots(figsize=(20, 8))
gold['Close'].plot(ax=ax, title='Gold - Preț de închidere (2005–2025)', legend=False)
ax.set_ylabel("Preț (USD)")
ax.grid(True)
ax.tick_params(axis='x', rotation=45)
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
fig.tight_layout()
plt.show()


fig, ax = plt.subplots(figsize=(20, 8))
gold_log_returns['Close'].plot(ax=ax, title='Gold - Preț de închidere (2005–2025)', legend=False)
ax.set_ylabel("Preț (USD)")
ax.grid(True)
ax.tick_params(axis='x', rotation=45)
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
fig.tight_layout()
plt.show()

In [ ]:
#Oil plot
fig, ax = plt.subplots(figsize=(20, 8))
oil['Close'].plot(ax=ax, title='Oil', legend=False)
ax.set_ylabel("Preț (USD)")
ax.grid(True)
ax.tick_params(axis='x', rotation=45)
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
fig.tight_layout()
plt.show()


#Oil log randamente
fig, ax = plt.subplots(figsize=(20, 8))
oil_log_returns['Close'].plot(ax=ax, title='Oil', legend=False)
ax.set_ylabel("Randament")
ax.grid(True)
ax.tick_params(axis='x', rotation=45)
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
fig.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# === SP500: Preț de închidere ===
fig, ax = plt.subplots(figsize=(20, 8))
sp500['Close'].plot(ax=ax, title='S&P 500 - Preț de închidere (2005–2025)', legend=False)
ax.set_ylabel("Preț (USD)")
ax.grid(True)
ax.tick_params(axis='x', rotation=45)
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
fig.tight_layout()
plt.show()

# === SP500: Randamente logaritmice ===
fig, ax = plt.subplots(figsize=(20, 8))
sp500_log_returns['Close'].plot(ax=ax, title='S&P 500 - Randamente zilnice (2005–2025)', legend=False)
ax.set_ylabel("Randament")
ax.grid(True)
ax.tick_params(axis='x', rotation=45)
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
fig.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# === ROTX: Preț de închidere ===
plt.figure(figsize=(20, 8))
rotx['Close'].plot(title='ROTX - Preț de închidere (2005–2025)')
plt.ylabel("Preț (USD)")
plt.grid(True)
plt.xticks(rotation=45)
plt.gca().xaxis.set_major_locator(mdates.YearLocator())  # un marcaj pe an
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.tight_layout()
plt.show()

# === ROTX: Randamente zilnice ===
plt.figure(figsize=(20, 8))
rotx_log_returns['Close'].plot(title='ROTX - Randamente zilnice (2005–2025)')
plt.ylabel("Randament")
plt.grid(True)
plt.xticks(rotation=45)
plt.gca().xaxis.set_major_locator(mdates.YearLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.tight_layout()
plt.show()

# **ADF/KPSS/PP**

In [ ]:
def analiza_trend(series, title=''):
    print(f'=== Analiza trend pentru {title} ===\n')

    # 2. Test ADF
    adf = ADF(series, trend='ct')
    print(adf.summary(),'\n')
    adf_stationary = adf.pvalue < 0.05

    # 3. Test KPSS
    kpss_test = KPSS(series, trend='ct')
    print(kpss_test.summary(),'\n')
    kpss_stationary = kpss_test.pvalue > 0.05

    # 4. Test Phillips-Perron
    pp = PhillipsPerron(series, trend='ct')
    print(pp.summary(),'\n')
    pp_stationary = pp.pvalue < 0.05

    # 5. Concluzie automată
    print('--- Concluzie ---')
    if adf_stationary and pp_stationary and kpss_stationary:
        print(f'Seria {title} are un trend determinist.')
    elif not adf_stationary and not pp_stationary and not kpss_stationary:
        print(f'Seria {title} are un trend stochastic.')
    else:
        print(f'Seria {title} necesită o analiză mai detaliată (rezultate mixte între teste).')
    print('=============================\n')

analiza_trend(sp500['Close'], title='S&P 500 - Close')
analiza_trend(sp500_log_returns['Close'], title='S&P 500 - Close')

In [ ]:
analiza_trend(rotx['Close'], title='ROTX - Close')
analiza_trend(rotx_log_returns['Close'], title='ROTX - Close')

In [ ]:
analiza_trend(gold['Close'], title='GOLD - Close')
analiza_trend(gold_log_returns['Close'], title='GOLD - Close')

In [ ]:
analiza_trend(oil['Close'], title='OIL - Close')
analiza_trend(oil_log_returns['Close'], title='OIL - Close')

In [ ]:
analiza_trend(bitcoin['Close'], title='BITCOIN - Close')
analiza_trend(bitcoin_log_returns['Close'], title='BITCOIN - Close')

# **ACF/PACF**

In [ ]:
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

def plot_acf_pacf(series, lags=20, title_prefix=''):
    """
    Afișează graficele ACF și PACF pentru o serie temporală dată.

    Parametri:
    - series: Pandas Series (ex: randamente log)
    - lags: număr de laguri pentru analiză
    - title_prefix: prefix pentru titlurile grafice (ex: 'S&P 500')
    """
    plt.figure(figsize=(10, 5))
    plot_acf(series, lags=lags, markersize=4)
    plt.title(f'ACF - {title_prefix}')
    plt.show()

    plt.figure(figsize=(10, 5))
    plot_pacf(series, lags=lags, markersize=4)
    plt.title(f'PACF - {title_prefix}')
    plt.show()

In [ ]:
plot_acf_pacf(sp500_log_returns['Close'], lags=20, title_prefix='S&P 500 Randamente')

In [ ]:
plot_acf_pacf(rotx_log_returns['Close'], lags=20, title_prefix='ROTX Randamente')

In [ ]:
plot_acf_pacf(gold_log_returns['Close'], lags=20, title_prefix='GOLD Randamente')

In [ ]:
plot_acf_pacf(oil_log_returns['Close'], lags=20, title_prefix='OIL Randamente')

In [ ]:
plot_acf_pacf(bitcoin_log_returns['Close'], lags=20, title_prefix='Bitcoin Randamente')

# **MODELE ARMA/ARIMA**

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from scipy.stats import kurtosis, skew
import numpy as np
import pandas as pd
import warnings

def testeaza_modele_arima(serie, p_max=3, d=0, q_max=3, alpha=0.05):
    rezultate = []

    for p in range(p_max + 1):
        for q in range(q_max + 1):
            try:
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore")

                    model = ARIMA(serie, order=(p, d, q))
                    fitted = model.fit()
                    resid = fitted.resid.dropna()

                    p_values = fitted.pvalues
                    nr_semnificativi = (p_values < alpha).sum()

                    rezultate.append({
                        'Model': f'ARIMA({p},{d},{q})',
                        'AIC': fitted.aic,
                        'BIC': fitted.bic,
                        'HQIC': fitted.hqic,
                        'LogLik': fitted.llf,
                        'Num_Params': len(fitted.params),
                        'Coef_Signif': nr_semnificativi,
                        'Sigma2': np.var(resid, ddof=1),
                        'Skewness': skew(resid),
                        'Kurtosis': kurtosis(resid),
                    })

            except Exception:
                continue

    df_rezultate = pd.DataFrame(rezultate)
    df_rezultate = df_rezultate.sort_values(by='AIC')
    return df_rezultate

testeaza_modele_arima(sp500_log_returns['Close'])

In [ ]:
best_model_sp500_arima = ARIMA(sp500_log_returns['Close'], order=( 2, 0, 1))
best_fitted_sp500 = best_model_sp500_arima.fit()
print(best_fitted_sp500.summary())

r_sp500 = best_fitted_sp500.resid
rsq_sp500 = r_sp500**2
rsq_sp500.head()

In [ ]:
testeaza_modele_arima(rotx_log_returns['Close'])

In [ ]:
best_model_rotx_arima = ARIMA(rotx_log_returns['Close'], order=( 3, 0, 3))
best_fitted_rotx = best_model_rotx_arima.fit()
print(best_fitted_rotx.summary())

r_rotx = best_fitted_rotx.resid
rsq_rotx = r_rotx**2
rsq_rotx.head()

In [ ]:
testeaza_modele_arima(gold_log_returns['Close'])

In [ ]:
best_fitted_gold_arima = ARIMA(gold_log_returns['Close'], order=( 1, 0, 1))
best_fitted_gold = best_fitted_gold_arima.fit()
print(best_fitted_gold.summary())

#Reziduuri normale si la patrat
r_gold = best_fitted_gold.resid
rsq_gold = r_gold**2
rsq_gold.head()

In [ ]:
testeaza_modele_arima(oil_log_returns['Close'])

In [ ]:
best_model_oil_arima = ARIMA(oil_log_returns['Close'], order=( 2, 0, 3))
best_fitted_oil = best_model_oil_arima.fit()
print(best_fitted_oil.summary())

r_oil = best_fitted_oil.resid
rsq_oil = r_oil**2
rsq_oil.head()

In [ ]:
testeaza_modele_arima(bitcoin_log_returns['Close'])

In [ ]:
best_model_bitcoin_arima = ARIMA(bitcoin_log_returns['Close'], order=( 3, 0, 2))
best_fitted_bitcoin = best_model_bitcoin_arima.fit()
print(best_fitted_bitcoin.summary())

r_bitcoin = best_fitted_bitcoin.resid
rsq_bitcoin = r_bitcoin**2
rsq_bitcoin.head()

# **TESTE ARCH-LM SI IPOTEZE FUNDAMENTALE**

In [ ]:
from statsmodels.stats.diagnostic import het_arch

def arch_lm_test(serie, nlags=10, titlu=None):
    """
    Aplică testul ARCH-LM (Engle) pentru heteroscedasticitate condiționată.

    Parametri:
    - serie: pd.Series cu randamente sau reziduuri
    - nlags: numărul de laguri pentru test (default: 10)
    - titlu: nume opțional pentru afișare

    Returnează:
    - dict cu valorile statistice
    """
    serie = serie.dropna()
    lm_stat, lm_pvalue, f_stat, f_pvalue = het_arch(serie, nlags=nlags)

    print("\n=== Test ARCH-LM (Engle) ===")
    if titlu:
        print(f"Serie: {titlu}")
    print(f"LM Statistic     : {lm_stat:.4f}")
    print(f"LM p-value       : {lm_pvalue:.4f}")
    print(f"F Statistic      : {f_stat:.4f}")
    print(f"F p-value        : {f_pvalue:.4f}")

    # === Concluzie automată ===
    print("-----------------------------------")
    if lm_pvalue < 0.05 or f_pvalue < 0.05:
        print("Rezultat: Se respinge H₀ ⇒ există heteroscedasticitate (ARCH efect detectat)")
        print("Recomandare: Model GARCH este justificat.")
    else:
        print("Rezultat: Nu se respinge H₀ ⇒ nu există dovezi de heteroscedasticitate")
        print("Recomandare: Varianța reziduurilor pare constantă.")
    print("===================================")


arch_lm_test(r_bitcoin, nlags=12, titlu="Bitcoin")
arch_lm_test(r_oil, nlags=12, titlu="Oil")
arch_lm_test(r_gold, nlags=12, titlu="Gold")
arch_lm_test(r_rotx, nlags=12, titlu="ROTX")
arch_lm_test(r_sp500, nlags=12, titlu="S&P 500")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

def show_gof_tests_residuals(residuals, titlu="Reziduuri"):
    """
    Vizualizează histograma + aplică 3 teste de normalitate:
    - Kolmogorov-Smirnov
    - Anderson-Darling
    - Cramér-von Mises

    Parametri:
    - residuals: pd.Series sau array-like
    - titlu: titlul graficului și al testelor
    """
    data = pd.Series(residuals).dropna()
    z = (data - np.mean(data)) / np.std(data)

    # === Histograma + overlay normal ===
    plt.figure(figsize=(8, 5))
    sns.histplot(data, bins=30, kde=False, stat='percent', color='lightgray')
    xmin, xmax = plt.xlim()
    x = np.linspace(xmin, xmax, 100)
    p = stats.norm.pdf(x, np.mean(data), np.std(data))
    plt.plot(x, p * 100 * (xmax - xmin) / 30, 'b-', label='Normal Fit')

    plt.title(f"Distribuția Reziduurilor – {titlu}")
    plt.xlabel("Reziduuri")
    plt.ylabel("Procent")
    plt.legend()
    plt.grid(True)
    plt.show()

    # === Teste GoF ===
    ks_stat, ks_p = stats.kstest(z, 'norm')
    ad_result = stats.anderson(data, dist='norm')
    cvm_result = stats.cramervonmises(data, 'norm')

    # === Output ===
    print(f"\nGoodness-of-Fit Tests pentru Reziduuri – {titlu}")
    print(f"Kolmogorov-Smirnov:     D = {ks_stat:.6f}, p = {ks_p:.4g}")
    print(f"Cramér-von Mises:       W² = {cvm_result.statistic:.6f}, p ≈ {cvm_result.pvalue:.3g}")
    print(f"Anderson-Darling:       A² = {ad_result.statistic:.6f}")
    print(f"Valori critice A-D      : {ad_result.critical_values}")

    # === Interpretare sumară ===
    if ks_p < 0.05 or ad_result.statistic > ad_result.critical_values[2]:
        print("Rezultatul sugerează abatere semnificativă de la normalitate.")
    else:
        print("Rezultatul nu indică abateri semnificative de la normalitate.")

show_gof_tests_residuals(r_bitcoin, titlu="Bitcoin")
show_gof_tests_residuals(r_oil, titlu="Oil")
show_gof_tests_residuals(r_gold, titlu="Gold")
show_gof_tests_residuals(r_rotx, titlu="ROTX")
show_gof_tests_residuals(r_sp500, titlu="S&P 500")

# **GARCH FITTING**

In [ ]:
from arch import arch_model
import pandas as pd

def compare_garch_models(series, p_range=range(1, 5), q_range=range(1, 5),
                          mean_type='Zero', show_summary=False):

    series = series.dropna()
    results = []

    vol_models = ['GARCH', 'EGARCH', 'FIGARCH']
    distributions = ['normal', 't', 'skewt', 'ged']

    for vol in vol_models:
        for dist in distributions:
            for p in p_range:
                for q in q_range:
                    try:
                        model = arch_model(series, mean=mean_type, vol=vol, p=p, q=q, dist=dist)
                        fitted = model.fit(disp='off')

                        params = fitted.params
                        pvalues = fitted.pvalues
                        total_params = len(params)
                        significant = {k: v for k, v in pvalues.items() if v < 0.05}

                        results.append({
                            'Model Volatilitate': vol,
                            'Distribuție': dist,
                            'GARCH(p,q)': f'({p},{q})',
                            'AIC': fitted.aic,
                            'BIC': fitted.bic,
                            'LogLik': fitted.loglikelihood,
                            'Nr. Parametri': total_params,
                            'Parametri Semnificativi': len(significant),
                            'Coef. Semnificativi': list(significant.keys())
                        })

                        if show_summary:
                            print(f'Model: {vol}, dist: {dist}, p: {p}, q: {q}')
                            print(fitted.summary())

                    except Exception as e:
                        print(f"Eroare {vol}({p},{q}) [{dist}]: {e}")
                        continue

    df_results = pd.DataFrame(results).sort_values(by='AIC').reset_index(drop=True)
    return df_results


compare_garch_models(r_sp500, p_range=range(1,5), q_range=range(1,5), show_summary=False)

Eroare FIGARCH(1,2) [normal]: p and q must be either 0 or 1.
Eroare FIGARCH(1,3) [normal]: p and q must be either 0 or 1.
Eroare FIGARCH(1,4) [normal]: p and q must be either 0 or 1.
Eroare FIGARCH(2,1) [normal]: p and q must be either 0 or 1.
Eroare FIGARCH(2,2) [normal]: p and q must be either 0 or 1.
Eroare FIGARCH(2,3) [normal]: p and q must be either 0 or 1.
Eroare FIGARCH(2,4) [normal]: p and q must be either 0 or 1.
Eroare FIGARCH(3,1) [normal]: p and q must be either 0 or 1.
Eroare FIGARCH(3,2) [normal]: p and q must be either 0 or 1.
Eroare FIGARCH(3,3) [normal]: p and q must be either 0 or 1.
Eroare FIGARCH(3,4) [normal]: p and q must be either 0 or 1.
Eroare FIGARCH(4,1) [normal]: p and q must be either 0 or 1.
Eroare FIGARCH(4,2) [normal]: p and q must be either 0 or 1.
Eroare FIGARCH(4,3) [normal]: p and q must be either 0 or 1.
Eroare FIGARCH(4,4) [normal]: p and q must be either 0 or 1.
Eroare FIGARCH(1,2) [t]: p and q must be either 0 or 1.
Eroare FIGARCH(1,3) [t]: p an

,Model Volatilitate,Distribuție,"GARCH(p,q)",AIC,BIC,LogLik,Nr. Parametri,Parametri Semnificativi,Coef. Semnificativi
0,GARCH,skewt,"(1,3)",-33659.445947,-33613.678746,16836.722974,7,5,"[omega, alpha[1], beta[1], eta, lambda]"
1,FIGARCH,skewt,"(1,1)",-33655.129998,-33615.900969,16833.564999,6,5,"[omega, d, beta, eta, lambda]"
2,GARCH,skewt,"(3,3)",-33652.086876,-33593.243332,16835.043438,9,6,"[omega, alpha[1], alpha[2], alpha[3], eta, lam..."
3,EGARCH,skewt,"(2,1)",-33649.819507,-33610.590478,16830.909754,6,6,"[omega, alpha[1], alpha[2], beta[1], eta, lambda]"
4,EGARCH,skewt,"(3,1)",-33648.072389,-33602.305187,16831.036194,7,6,"[omega, alpha[1], alpha[2], beta[1], eta, lambda]"
...,...,...,...,...,...,...,...,...,...
127,GARCH,normal,"(1,3)",-33246.980993,-33214.290135,16628.490497,5,5,"[omega, alpha[1], beta[1], beta[2], beta[3]]"
128,EGARCH,normal,"(3,4)",-33246.800421,-33194.495048,16631.400210,8,1,[alpha[1]]
129,GARCH,normal,"(3,4)",-33246.464655,-33194.159282,16631.232327,8,1,[omega]
130,GARCH,normal,"(4,4)",-33234.955011,-33176.111466,16626.477505,9,2,"[omega, alpha[1]]"


In [ ]:
compare_garch_models(r_oil, show_summary=True)

In [ ]:
compare_garch_models(r_bitcoin, show_summary=True)

In [ ]:
compare_garch_models(r_gold, show_summary=True)

In [ ]:
compare_garch_models(r_rotx, show_summary=True)

MERGE BITCOIN

In [ ]:
import pandas as pd
import glob
import os

# Calea către folderul în care sunt fișierele
folder = "/content/aladam"

# Obține toate fișierele CSV din folder, indiferent de nume
csv_files = glob.glob(os.path.join(folder, "*.csv"))

# Inițializează o listă pentru a stoca fiecare DataFrame
dataframes = []

# Parcurge fiecare fișier și încarcă-l cu separatorul ;
for file in csv_files:
    try:
        df = pd.read_csv(file, sep=';')
        dataframes.append(df)
    except Exception as e:
        print(f"Eroare la citirea fișierului {file}: {e}")

# Concatenează toate datele într-un singur DataFrame
df_merged = pd.concat(dataframes, ignore_index=True)

# Conversie la datetime și sortare după 'timeOpen'
if 'timeOpen' in df_merged.columns:
    df_merged['timeOpen'] = pd.to_datetime(df_merged['timeOpen'], errors='coerce')
    df_merged = df_merged.sort_values('timeOpen')

# Salvează rezultatul într-un fișier CSV
df_merged.to_csv("bitcoin_merged_all_years.csv", index=False)

print("Fișierele au fost îmbinate și sortate cu succes.")
